In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz

# 定义函数 f(x, M, G, F)

In [2]:
def f(x, M, G, F):
    return F**2 * ((M**4 + M**2 * G**2) / (24 * np.pi * ((x - M**2)**2 + M**2 * G**2)))

# 设置参数和网格

In [3]:
X1, X2 = 0, 2
Y1, Y2 = X2 + 0.01, 30 + 0.01
n = int((X2 - X1) * 1000 + 1)
m = int((Y2 - Y1) * 100 + 1)
hx = (X2 - X1) / (n - 1)
hy = (Y2 - Y1) / (m - 1)
x = np.linspace(X1, X2, n)
y = np.linspace(Y1, Y2, m)

# 定义参数和函数


In [4]:
params = [
    (0.22, 0.78, 0.15),
    (0.19, 1.46, 0.4),
    (0.14, 1.72, 0.25),
    (0.14, 1.90, 0.1)
]

F = np.zeros_like(x)
for param in params:
    fV, m_, gamma = param
    F += f(x, m_, gamma, 1)


# 计算 G1 和 G2


In [5]:
G1 = np.zeros((m, 1))
G2 = np.zeros((m, 1))
for i in range(m):
    G1[i, 0] = trapz(1 / (y[i] - x) * F, x)
    G2[i, 0] = trapz(1 / (y[i] - x) * (x - x), x)  # 注意这里 f2 的定义

# 设置误差参数


In [6]:
B1, B2 = 1, 2
XD = 1.01
b1, b2 = XD * B1, XD * B2

# 计算 f_exact 和 f_delta


In [7]:
f_exact = B1 * F + B2 * (x - x)
f_delta = b1 * F + b2 * (x - x)

# 计算 g_exact 和 g_delta


In [8]:
g_exact = B1 * G1 + B2 * G2
g_delta = b1 * G1 + b2 * G2

# 计算 V_exact

In [9]:
MM = f_delta[0]
NN = f_delta[-1]
V_exact = (MM * (x - X2)) / (X1 - X2) + (NN * (x - X1)) / (X2 - X1)
G3 = np.zeros((m, 1))
for i in range(m):
    G3[i, 0] = trapz(1 / (y[i] - x) * V_exact, x)

U_exact = f_exact - V_exact
G_exact = g_exact - G3
G_delta = g_delta - G3

# 构建矩阵 A 和 Aphi


In [10]:
A = np.zeros((n, n))
Aphi = np.zeros((m, n))
for j in range(1, n - 1):
    Aphi[:, j] = 1 / hx * ((y - x[j - 1]) * np.log(y - x[j - 1]) + (x[j - 1] - x[j]) +
                           (y - x[j + 1]) * np.log(y - x[j + 1]) - 2 * (y - x[j]) * np.log(y - x[j]) - (x[j] - x[j + 1]))

Aphi[:, 0] = np.log(y - x[0]) + 1 / hx * ((y - x[1]) * np.log(y - x[1]) - (y - x[0]) * np.log(y - x[0]) - (x[0] - x[1]))
Aphi[:, -1] = -np.log(y - x[-1]) + 1 / hx * (-(y - x[-1]) * np.log(y - x[-1]) + (y - x[-2]) * np.log(y - x[-2]) + (x[-2] - x[-1]))

# 初始化 ba 数组


In [11]:
ba=[]

# 构建矩阵 A 和向量 ba


In [12]:
for i in range(n):
    for j in range(n):
        A[i, j] = trapz(Aphi[:, i] * Aphi[:, j], y)
    ba.append(trapz(Aphi[:, i] * G_delta, y))
    # print(ba[i])

In [13]:
ba

[array([-1.95118710e-04, -1.85580957e-04, -1.77706042e-04, ...,
        -1.47049504e-06, -1.46995765e-06, -1.46942065e-06]),
 array([-3.90282142e-04, -3.71204451e-04, -3.55452815e-04, ...,
        -2.94132714e-06, -2.94025223e-06, -2.93917810e-06]),
 array([-3.90349150e-04, -3.71268183e-04, -3.55513843e-04, ...,
        -2.94183214e-06, -2.94075704e-06, -2.93968273e-06]),
 array([-3.90416268e-04, -3.71332021e-04, -3.55574971e-04, ...,
        -2.94233796e-06, -2.94126268e-06, -2.94018819e-06]),
 array([-3.90483415e-04, -3.71395886e-04, -3.55636126e-04, ...,
        -2.94284402e-06, -2.94176855e-06, -2.94069387e-06]),
 array([-3.90550533e-04, -3.71459723e-04, -3.55697254e-04, ...,
        -2.94334984e-06, -2.94227419e-06, -2.94119933e-06]),
 array([-3.90617756e-04, -3.71523660e-04, -3.55758478e-04, ...,
        -2.94385646e-06, -2.94278063e-06, -2.94170557e-06]),
 array([-3.90685018e-04, -3.71587634e-04, -3.55819737e-04, ...,
        -2.94436337e-06, -2.94328735e-06, -2.94221212e-06]),


# 构建矩阵 M 和 M1


In [14]:
M = np.zeros((n, n))
M[0, 0] = 4
M[0, 1] = 2
for i in range(1, n - 1):
    M[i, i] = 8
    M[i, i - 1] = 2
    M[i, i + 1] = 2
M[-1, -1] = 4
M[-1, -2] = 2
M = hx / 12 * M

M1 = np.zeros((n, n))
M1[0, 0] = 1
M1[0, 1] = -1
for i in range(1, n - 1):
    M1[i, i] = 2
    M1[i, i - 1] = -1
    M1[i, i + 1] = -1
M1[-1, -1] = 1
M1[-1, -2] = -1
M1 = 1 / hx * M1

H1 = M + M1

# 删除多余的行和列


In [15]:
A = np.delete(A, [0, n - 2], axis=0)
A = np.delete(A, [0, n - 2], axis=1)
ba = np.delete(ba, [0, n - 2])
H1 = np.delete(H1, [0, n - 2], axis=0)
H1 = np.delete(H1, [0, n - 2], axis=1)

# 初始化变量


In [16]:
ite = 20
alpha = np.zeros(ite)
ua = np.zeros((n - 2, ite))
Ua = np.zeros((n, ite))
fa = np.zeros((n, ite))
res = np.zeros(ite)
err = np.zeros(ite)
L = np.zeros(ite)
LH = np.zeros(ite)

# 循环迭代


In [17]:
for k in range(ite):
    alpha[k] = 10**(-k)
    ua[:, k] = np.linalg.solve(A + alpha[k] * H1, ba)
    Ua[:, k] = np.concatenate(([0], ua[:, k], [0]))
    fa[:, k] = Ua[:, k] + V_exact
    res[k] = np.sqrt(trapz((Aphi @ fa[:, k] - g_delta)**2, y))
    err[k] = np.sqrt(trapz((fa[:, k] - f_exact)**2, x))
    L[k] = np.sqrt(trapz(x, fa[:, k]**2)) * res[k]
    LH[k] = np.sqrt(trapz(x, fa[:, k]**2 + (np.gradient(fa[:, k]))**2)) * res[k]

ValueError: solve1: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m)->(m) (size 5604799 is different from 1999)

# 寻找最小的 L 和 LH 值的索引


In [ ]:
kl = np.argmin(np.abs(L))
klh = np.argmin(np.abs(LH))

# 计算 GCV


In [ ]:
V = np.zeros(ite)
for i in range(ite):
    VV = np.eye(len(y)) - Aphi @ np.linalg.solve(Aphi.T @ Aphi + alpha[i] * np.eye(len(x)), Aphi.T)
    V1 = np.linalg.norm(VV @ g_delta)**2
    V2 = np.trace(VV)**2
    V[i] = V1 / V2

# 寻找最小的 GCV 值的索引


In [ ]:
Vi = np.argmin(V)

# 计算拟最优参数


In [ ]:
Rho = np.zeros(ite)
for i in range(ite):
    Rho[i] = np.linalg.norm(alpha[i] * (Aphi.T @ Aphi + alpha[i] * np.eye(len(x))) @ fa[:, i])


# 寻找最小的 Rho 值的索引

In [ ]:
Ri = np.argmin(Rho)

# 输出结果


In [ ]:
results = {
    'Method': ['L-curve+L^2', 'L-curve+H^1', 'GCV', '拟最优'],
    'Selected Parameter Index': [kl, klh, Vi, Ri]
}
print(results)

# 绘制图形


In [ ]:
plt.figure(figsize=(15, 4))
for k in range(ite):
    plt.subplot(4, 5, k + 1)
    plt.plot(x, f_exact, label='Exact', linewidth=2)
    plt.plot(x, fa[:, k], label=f'Alpha={alpha[k]:.1e}', linewidth=2)
    plt.title(f'Alpha = {alpha[k]:.1e}')
    plt.legend()
plt.tight_layout()
plt.show()